In [ ]:
DIR="../inputs/RootFiles/"
import ROOT
%jsroot on
canvas = ROOT.TCanvas("c1", "", 200, 10, 1000, 1000)
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetOptTitle(0)

# ... and More Functions

In [ ]:
lumi = {12:11580. , 11:2330.}


def get_histogram(file_dir, key, **hist_args):
    with ROOT.TFile.Open(f"{DIR}/{file_dir}") as file:
        hist = file.demo.Get(key)
        hist.SetDirectory(0)
        
    if "title" in hist_args:
        title = hist_args["title"]
        hist.SetTitle(title)
    
    if "scale" in hist_args:
        scale = hist_args["scale"]
        hist.Scale(scale)
    
    return hist



def histogram_stack(name, historams, last=False, ):
    stack = ROOT.THStack(name , "")
    for hist in historams:
        stack.Add(hist)
    
    return stack
    

def create_canvas(name, histogram, drawing_format, x_title=None, y_title=None, legend_title=None):
    canvas = ROOT.TCanvas(name)
    histogram.Draw(drawing_format)
    canvas.BuildLegend(0.7, 0.7, 0.95, 0.95 , legend_title, "f")
    
    if y_title is not None:
        ...  # TODO
    
    if x_title is not None:
        ...  # TODO
    
    return canvas

def ZZto4lep_hists():
    sfZZ = 1.386
    zz_xsections = {12:{'4e':0.077 , '4mu':0.077 , '2mu2e':0.18},
                    11:{'4e':0.067 , '4mu':0.067 , '2mu2e':0.15}}

    zz_totals = {12:{'4mu':1499064 , '4e':1499093 , '2mu2e':1497445},
                11:{'4mu':1447136 , '4e':1493308 , '2mu2e':1479879}}

    hists = []
    for decaymode in ['4mu' , '4e' , '2mu2e']:
        for year in [11,12]:

            file_directory  = f"20{year}/simulation/ZZ{decaymode}{year}.root"
            file_key        = f"mass{decaymode}_8TeV_low"
            scale_factor    = zz_xsections[year][decaymode]*lumi[year]*sfZZ / zz_totals[year][decaymode] 
            histogram_title = f"{decaymode.replace('mu' , '#mu')}, 20{year}"

            h = get_histogram(file_dir = file_directory,
                              key      = file_key,
                              scale    = scale_factor,
                              title    = histogram_title)
            hists.append(h)

    return hists

def TTbarto4lep_hists():
    tt_xsections = {12:200. , 11:177.31}
    tt_totals    = {12:6423106 , 11:9771205}
    sfTTBar      = {11:0.11 , 12:1 }

    hists = []

    for year in [11,12]:
        file_directory = f"20{year}/simulation/TTBar{year}.root"

        for decaymode in ['4mu' , '4e' , '2mu2e']:
            file_key          = f"mass{decaymode}_8TeV_low"
            scale_factor = tt_xsections[year]*lumi[year]*sfTTBar[year] / tt_totals[year]
            histogram_title        = "{0}, 20{1}".format( decaymode.replace('mu' , '#mu') ,year)

            current_hist = get_histogram(file_dir = file_directory,
                              key      = file_key,
                              scale    = scale_factor,
                              title    = histogram_title)

            hists.append(current_hist)

    return hists


def DrellYan_hists():
    dy_xsections = {12:{10:10.742 , 50:2955.} ,
                   11:{10:9507. , 50:2475.}}
    
    dy_totals    = {12:{10:6462290 , 50:29426492},
                    11:{10:39909640, 50:36408225}}
    
    sfDY = 1.12
    
    hists = []
    for mass in [10,50]:
        fname_extension = 'TuneZ' if mass == 50 else ''
        for year in [11,12]:
            file_directory = f"20{year}/simulation/DY{mass}{fname_extension}{year}.root"

            for decaymode in ['4mu' , '4e' , '2mu2e']:
                file_key = f"mass{decaymode}_8TeV_low"
                scale_factor = dy_xsections[year][mass]*lumi[year]*sfDY / dy_totals[year][mass]
                histogram_title = "DY({2}), {0}, 20{1}".format( decaymode.replace('mu' , '#mu') ,year , mass)

                current_hist = get_histogram(file_dir = file_directory,
                                             key      = file_key,
                                             scale    = scale_factor,
                                             title    = histogram_title)

                hists.append(current_hist)

    return hists
    
def HZZto4lep_hists():
    scalexsecHZZ = {12:0.0065 , 11:0.0057}
    nevtHZZ      = {12:299973 , 11:299683 }
    
    hists = []

    for y in [11,12]:
        file_directory  = "20{0}/simulation/HZZ{0}.root".format(y)
        
        for decaymode in ['4mu' , '4e' , '2mu2e']:
            
            file_key        = "mass{0}_8TeV_low".format(decaymode)
            scale_factor    = lumi[y]*scalexsecHZZ[y]/nevtHZZ[y]
            histogram_title = "{0}, 20{1}".format(decaymode , y)
            
            current_hist = get_histogram(file_dir  = file_directory,
                                          key      = file_key,
                                          title    = histogram_title,
                                          scale    = scale_factor)
            hists.append(current_hist)
    return hists

def Data4lep_hists():
    ROOT.gStyle.SetPalette(ROOT.kRainBow)
    allHists = []

    for lep in ['Mu' , 'E']:
        for year in [11,12]:
            file_directory  = "20{0}/data/Double{1}{0}.root".format(year , lep)
            file_key        = "mass4{0}_8TeV_low".format(lep.lower())
            histogram_title = "4{0}, 20{1}".format( '#mu' if lep=="Mu" else 'e',year)

            h = get_histogram(file_dir = file_directory,
                               key      = file_key,
                               title    = histogram_title)

            allHists.append(h)

            if lep=='Mu': #if the doubleMu file is opened, 2mu2e histogram should also be read
                file_key        = "mass2mu2e_8TeV_low"
                histogram_title = "2#mu+2e, 20{0}".format(year)
                h = get_histogram(file_dir = file_directory,
                                  key      = file_key,
                                  title    = histogram_title)
                allHists.append(h)
                
    return allHists

# Reading Background enevts

In [ ]:
DY_stack = histogram_stack("DY", DrellYan_hists())
ZZ_stack = histogram_stack("ZZ", ZZto4lep_hists())
TT_stack = histogram_stack("TT", TTbarto4lep_hists())

DY = DY_stack.GetStack().Last()
DY.SetLineColor(ROOT.kBlack)
DY.SetFillColor(ROOT.kGreen -5)
DY.SetLineWidth(2)
DY.SetLineStyle(1)

ZZ = ZZ_stack.GetStack().Last()
ZZ.SetLineColor(ROOT.kBlack)
ZZ.SetFillColor(ROOT.kAzure -9)
ZZ.SetLineWidth(2)
ZZ.SetLineStyle(1)

TT = TT_stack.GetStack().Last()
TT.SetLineColor(ROOT.kBlack)
TT.SetFillColor(ROOT.kGray)
TT.SetLineWidth(2)
TT.SetLineStyle(1)

background_events = [DY, TT, ZZ]

In [ ]:
simulation = histogram_stack("simulation", background_events)

In [ ]:
simulation.Draw('hist')
canvas.Draw()

# Reading the Signal

In [ ]:
HZZ_stack = histogram_stack("H", HZZto4lep_hists())
    
signal = HZZ_stack.GetStack().Last()

signal.SetLineColor(ROOT.kRed)
signal.SetLineWidth(2)
signal.SetLineStyle(1)
signal.SetFillStyle(0)
signal.SetFillColor(0)

In [ ]:
signal.Draw("Hist")
canvas.Draw()

# Reading the Data

In [ ]:
Data_4l_stack = histogram_stack("Data4lep", Data4lep_hists())
Data_4l = Data_4l_stack.GetStack().Last()

# Formatting 
Data_4l.SetMarkerColor(ROOT.kBlack)
Data_4l.SetMarkerStyle(20)
Data_4l.SetMarkerSize(1.7)
Data_4l.SetLineColor(ROOT.kBlack)
Data_4l.SetLineWidth(1)
Data_4l.Draw("PE1")


In [ ]:
Data_4l.Draw("PE1")
canvas.Draw()

# Simulation: Background + Signal

In [ ]:
simulation.Add(signal)
simulation.Draw("Hist")
canvas.Draw()

# Compute $\chi^2$ Statistic and p-value with `Chi2Test` Method

In [ ]:
Data_4l.Chi2Test(simulation.GetStack().Last(), "Chi2 UW P")  # Chi2

In [ ]:
Data_4l.Chi2Test(simulation.GetStack().Last(), "UW P")  # p-value

# Minimizing the $\chi^2$ statistic

In [ ]:
import scipy.optimize as opt

In [ ]:
# TODO: Complete the function

def ChiSquareStatistic(signal_strength, test='Chi2 UW'):
    # Scaling the signal
    signal.Scale(...)
    
    # Combine all events; Background + Signal
    sim = ...
    
    # Compute Chi-square statistic
    chi2 = Data_4l.Chi2Test(...)
    
    # Important: Re-scaling the signal to keep it unchanged
    ...
    
    return chi2


def ChiSquareMinimization(initial_guess=0.5):
    res = opt.minimize(ChiSquareStatistic, x0=initial_guess)
    
    if res.success:
        
        print("================================")
        print(f" Chi-Square Statistic: {res.fun:0.05f}")
        print(f" Signal Intesity: {res.x[0]:0.05f}")
        print("================================")
        
        return res.x, res.fun
    
    else:
        print("================================")
        print("============Failed==============")
        print("================================")
    


In [ ]:
ss = ChiSquareMinimization()[0][0]
p_value = ChiSquareStatistic(ss,'UW')

# Visualization the $\chi^2$ w.r.t Signal Strength

In [ ]:
import numpy as np
import plotly.express as px


signal_strength = np.linspace(0.1, 2, 100)
chi2_array      = [ChiSquareStatistic(ss) for ss in signal_strength]

fig = px.scatter(x=signal_strength, y=chi2_array)
fig.add_annotation(x=ss, y=ChiSquareStatistic(ss),
            text="Best signal strength",
            showarrow=True,
            arrowhead=1)

fig.update_xaxes(title=r"$\frac{\sigma}{\sigma_{SM}}$")
fig.update_yaxes(title=r"$\chi^2$");

In [ ]:
fig.show()

# Final Result

In [ ]:
HZZ_stack = histogram_stack("H", HZZto4lep_hists())
    
signal = HZZ_stack.GetStack().Last()

signal.SetLineColor(ROOT.kRed)
signal.SetLineWidth(2)
signal.SetLineStyle(1)
signal.SetFillStyle(0)
signal.SetFillColor(0)

Signal_scaled = False

In [ ]:
if not Signal_scaled:
    signal.Scale(ss)
    Signal_scaled = True
    
simulation = histogram_stack(f"sim", [*background_events, signal])

leg = ROOT.TLegend(.62, .70, .82, .88)
leg.SetFillColor(0)
leg.SetBorderSize(0)
leg.SetTextFont(42)
leg.SetTextSize(0.038)

leg.AddEntry(Data_4l, "Data", "PE1")
leg.AddEntry(DY, "Z/#gamma* + X", "f")
leg.AddEntry(TT, "TTBar", "f")
leg.AddEntry(ZZ, "ZZ -> 4l", "f")
leg.AddEntry(signal, "m_{H} = 125 GeV", "f")

simulation.Draw("hist")
Data_4l.Draw("PE1 same")

simulation.GetXaxis().SetTitle("m_{4l} (GeV)")
simulation.GetYaxis().SetTitle("Events / 3 GeV")
simulation.GetYaxis().SetTitleOffset(1.2)
simulation.SetMaximum(30)

txt1 = ROOT.TLatex()
txt1.DrawTextNDC(0.15,0.84,f"CMS Open Data")
txt1.DrawText(0.15,0.7,f"p-value = {p_value:0.2f}")
txt1.SetTextSize(0.03)
txt1.SetNDC()
txt1.SetTextFont(42)
txt1.DrawLatex(0.14, 0.91, "#sqrt{s} = 7 TeV, L = 2.3 fb^{-1}, #sqrt{s} = 8 TeV, L = 11.6 fb^{-1}" )

leg.Draw()
canvas.Draw()